In [99]:
from requests import get
from bs4 import BeautifulSoup
from datetime import datetime
from pathlib import Path
import json
import pandas as pd

def write_json(self, data):
    self.write_text(json.dumps(data))
    return True
    
def read_json(self):
    return json.loads(self.read_text())
    
Path.write_json = write_json
Path.read_json = read_json

In [100]:
urls = {
    'uk': 'https://www.timeanddate.com/holidays/uk/2022?hol=9',
    'singapore': 'https://www.timeanddate.com/holidays/singapore/2022?hol=9',
    'brazil': 'https://www.timeanddate.com/holidays/brazil/2022?hol=9',
    'us': 'https://www.timeanddate.com/holidays/us/2022?hol=9',
    'germany': 'https://www.timeanddate.com/holidays/germany/2022?hol=9',
    'italy': 'https://www.timeanddate.com/holidays/italy/2022?hol=9',
    'france': 'https://www.timeanddate.com/holidays/france/2022?hol=9',
    'russia': 'https://www.timeanddate.com/holidays/russia/2022?hol=9',
    'china': 'https://www.timeanddate.com/holidays/china/2022?hol=9',
    'south-korea': 'https://www.timeanddate.com/holidays/south-korea/2022?hol=9',
    'japan': 'https://www.timeanddate.com/holidays/japan/2022?hol=9',
    'india': 'https://www.timeanddate.com/holidays/india/2022?hol=134217737',
}

In [104]:
df = pd.DataFrame()

for country, url in urls.items():
    datafile = Path(f'{country}.json')
    if datafile.exists():
        data = datafile.read_json()
        for datum in data:
            day_name = datum['day_name']
            s = pd.Series(datum, name=day_name)
            df = df.append(s)
        continue
        
    htmlfile = Path(f'{country}.html')
    if not htmlfile.exists():
        r = get(url)
        htmlfile.write_text(r.text)
    
    html = htmlfile.read_text()
    
    soup = BeautifulSoup(html, 'lxml')
    
    table = soup.find('table', {'id': 'holidays-table'})
    
    data = []
    for tr in table.tbody.find_all('tr'):
        if tr.get('data-date'):
            timestamp = tr.get('data-date')
            timestamp = timestamp[:-3]
            dt_object = datetime.fromtimestamp(int(timestamp))
        if len(tr.find_all('td')) == 4:
            _, day, day_name, observance = [x.text for x in tr.find_all('td')]
        elif len(tr.find_all('td')) == 3:
            day, day_name, observance = [x.text for x in tr.find_all('td')]
            
        datum = {
            'country': country,
            'date': dt_object.strftime('%Y-%m-%d'),
            'day': day,
            'day_name': day_name,
            'observance': observance
        }
        
        data.append(datum)
        
        s = pd.Series(datum, name=day_name)
        
        df = df.append(s)
            
    datafile.write_json(data)

In [93]:
df.to_csv('dates.csv')